In [5]:
import chardict
import drawBot as db
import logging
import os

from chardict import all_script_typefaces, all_typefaces_fonts, all_typefaces

In [6]:
# generate samples to use in the study
# takes long! (16m)

def make_study_samples(c, path, font, fontsize=1., fontshift=0, script="latin", size=250, format="pdf",
                       characters=[], background_colour=None,
                       outline=False):
    """
    Export samples to use in the study.
    """

    # draw character sample
    w = size

    db.newPage(w, w)
    if background_colour:
        db.fill(*background_colour)
        db.rect(0, 0, w, w)
    if outline:
        db.stroke(0.5)
        db.fill(None)
        db.rect(0, 0, w, w)
    db.fill(0)
    db.stroke(None)
    db.font(font)
    if script == "devanagari":
        # inteded character height is ca 3/5 of the square height
        # precisely, it is 160px ca 64 % (see metrics below)
        # changing the constant 0.6 here would be pain,
        # so I am keeping it
        fontsize = w * 0.6 * fontsize
        fontshift = fontsize * fontshift * 1.7
        db.fontSize(fontsize)
        wt, _ = db.textSize(c)
        x = (w - wt) / 2
        #y = size * 0.8 + fontshift
        y = size * 1.2 + fontshift
        db.text(c, (x, y))
        # metrics guidelines (for visual check)
        # db.stroke(1,0,0)
        # db.line((0, 60), (size, 60))
        # db.line((0, 200), (size, 200))
        # db.stroke(None)
    else:
        # intended x-height, around 1/3 of the height of the square
        x_height = size * 0.35
        # change size so that all x-heights are the same
        db.fontSize(1000)
        fontsize = 1000 / db.fontXHeight() * x_height
        db.fontSize(fontsize)
        wt, _ = db.textSize(c)
        x = (w - wt) / 2
        y = (w - x_height) / 2
        db.text(c, (x, y))

    db.saveImage(path, multipage=False)

for script in all_script_typefaces.keys():
    print(script)
    for group in all_script_typefaces[script].keys():
        for typeface in all_script_typefaces[script][group]:
            print("-", typeface)
            pdfdir = os.path.join("graphics", "study-samples", script, typeface)
            if not os.path.exists(pdfdir):
                os.makedirs(pdfdir)
            if all_typefaces_fonts[typeface][0] not in db.installedFonts():
                logging.error("Representative font is not available.")
                continue
            for cn, (_, sample) in chardict.chardict[script].items():
                pdfpath = os.path.join(pdfdir, "%s.pdf" % cn)
                make_study_samples(sample, pdfpath, *all_typefaces_fonts[typeface], script,
                                   size=250, background_colour=(1, 1, 1), outline=True)
